**Notebook Overview**

This notebook is designed to build and deploy a predictive model for the Urban Heat Island (UHI) Index using an extensive set of engineered features. The inputs to this notebook are the feature-rich datasets created in a separate preprocessing notebook. These datasets contain detailed environmental, urban, and meteorological features, as well as spatial and temporal derivatives generated through various advanced data integration and feature engineering techniques.

**Key Steps and Processes:**

1. **Data Loading and feature interactions:**

   - The processed training and submission datasets are loaded and joined, additional interaction features are added afterwards to these datasets.

2. **Model Optimization (Optional):**

   - A Bayesian optimization routine (using Optuna) was used to tune the hyperparameters of an XGBoost regression model. The best parameters are saved for later use.

3. **Model Training & Evaluation:**

   - The optimized model is retrained on the full training dataset.
   - The model is evaluated on a test split, and performance metrics (such as R²) are calculated to assess its predictive power.

4. **Prediction Generation:**
   - The trained model is used to predict the UHI Index for the submission dataset.
   - The predictions are then formatted into a submission file using a predefined template.

**Outputs:**

- **Trained Model:**  
  An XGBoost regression model that has been optimized and retrained using the full training dataset (The models parameters can be saved).

- **Prediction File:**  
  A submission file containing the predicted UHI Index values for the submission dataset.


In [3]:
import os
import json
import warnings

# Data manipulation
import numpy as np
import pandas as pd
import gdown
# Geospatial and projections
from scipy.spatial import cKDTree

# Modeling and machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.neighbors import KernelDensity

import xgboost as xgb
#import optuna

# Disable warnings
warnings.filterwarnings("ignore")





In [ ]:
# Step 3: Set the folder ID and output directory
folder_url = "https://drive.google.com/drive/folders/1gMzVpTQ_YBMVWkQZXSFFNqOCAkvHehRN?usp=drive_link"
folder_id = "1gMzVpTQ_YBMVWkQZXSFFNqOCAkvHehRN"
output_dir = "./"

# Step 4: Download the entire folder
gdown.download_folder(id=folder_id, output=output_dir, quiet=False, use_cookies=False)

# Step 5: List downloaded files
os.listdir(output_dir)

Retrieving folder contents


Retrieving folder 1X9akY68NZtdZwYJsH19UqyRLQENx__oD data_processed
Processing file 1oE6rSyCmimDtovW4pi-j9HRieljt3bZ2 01_data_satellite_submission.pkl
Processing file 1h7MRZNy5cierKCMvZO_--guqCa63wYaR 01_data_satellite_train.pkl
Processing file 1MZ61PF1HLFdYzkkiEgVzckcSCuhm74Fu 02_data_weather_submission.pkl
Processing file 1G9cAX6nrFBvDYZVIkIvS2G8ilOnRxURJ 02_data_weather_train.pkl
Processing file 1Ae8hCwoMqRqcr4_oqn9sSuxCQG49TiKQ 03_data_footprint_submission.pkl
Processing file 1hjurV6wMgAKv6dM8kM5-qhxkguFh2yuf 03_data_footprint_train.pkl
Processing file 1SqxocwIb2XG7KO7nKvBKXwEq_hhBP-Dh 04_data_elevation_submission.pkl
Processing file 1BAyCAiQwtl1oy9kWRvc2oJUG3NdEUruI 04_data_elevation_train.pkl
Processing file 158rNXE9WmOM5vNiV2kGXfrnxHvM79zbJ best_model_params.json
Processing file 1BJWtmzyzqfSYSKLCLsC1Su2p17JzllPE my_xgb_model.bin
Processing file 1uAwvh3gB9RnQQuGL_2CKb27lMlm6znvC Submission_template.csv
Processing file 1DOtWrz_skMpQ9goRiW8SbXtrko_sdCq_ Submission.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed


PermissionError: [Errno 13] Permission denied: '/content'

# Data Extraction and Preparation


## Loading features


The following dataframes are generated in a separate notebook, where features are extracted and processed from multiple data sources. These features include:

- **Satellite Data:** Information derived from remote sensing, such as land surface temperature, vegetation indices ...
- **Buildings Data:** Structural and urbanization-related features, including building density, height, area ...
- **Weather Data:** Meteorological variables such as temperature, humidity, wind speed, and other atmospheric conditions.
- **Existing Features:** The original features present in the training and submission datasets, including **Longitude, Latitude, and UHI Index**.

These processed datasets are then combined into unified training and submission dataframes to be used for model training and predictions.


In [3]:
# importing satellite, buildings, weather and elevation features for training data
processed_train_sat = pd.read_pickle(os.path.join("data", "data_processed", "01_data_satellite_train.pkl"))
processed_train_weather = pd.read_pickle(os.path.join("data", "data_processed", "02_data_weather_train.pkl"))
processed_train_buildings = pd.read_pickle(os.path.join("data", "data_processed", "03_data_footprint_train.pkl"))
processed_train_elevation = pd.read_pickle(os.path.join("data", "data_processed", "04_data_elevation_train.pkl"))


# importing satellite, buildings, weather and elevation features for submission data
processed_sub_sat = pd.read_pickle(os.path.join("data", "data_processed", "01_data_satellite_submission.pkl"))
processed_sub_weather = pd.read_pickle(os.path.join("data", "data_processed", "02_data_weather_submission.pkl"))
processed_sub_buildings = pd.read_pickle(os.path.join("data", "data_processed", "03_data_footprint_submission.pkl"))
processed_sub_elevation = pd.read_pickle(os.path.join("data", "data_processed", "04_data_elevation_submission.pkl"))


dfs_train = [processed_train_sat, processed_train_weather, processed_train_buildings , processed_train_elevation]
dfs_sub = [processed_sub_sat, processed_sub_weather, processed_sub_buildings , processed_sub_elevation]


# Alternatively, using a loop:
processed_train = dfs_train[0]
for df in dfs_train[1:]:
    processed_train = processed_train.merge(df, on=['Longitude', 'Latitude', 'datetime', 'UHI Index'])

processed_sub = dfs_sub[0]
for df in dfs_sub[1:]:
    processed_sub = processed_sub.merge(df, on=['Longitude', 'Latitude', 'UHI Index'])

## Interaction between features


This section of the code is dedicated to creating new features that enhance our Urban Heat Island (UHI) prediction model. The process involves several steps:

1. **Reference LST Calculation:**

   - **Purpose:**  
     The code first defines a reference location using a fixed longitude and latitude. It then extracts the median Land Surface Temperature (LST) from the training dataset at that reference location.
   - **Why:**  
     The reference LST serves as a baseline to compare the temperature at other points.

2. **Engineered Features Function (`engineer_new_features`):**  
   The function takes an input DataFrame and adds several new features based on the existing data. Key components include:

   - **Vegetation & Solar Features:**

     - **NDVI_solar:** Multiplies the median NDVI (vegetation index) by the 3–4 pm solar flux to capture shading effects.
     - **Effective Solar Flux:** Adjusts the solar flux by building density (if available) to reflect canopy cover or shading effects.

   - **Temperature & Humidity Interactions:**

     - **hum_temp:** Calculates a combined metric (a proxy for “feels like” temperature) by multiplying the 3–4 pm air temperature with the relative humidity (scaled appropriately).

   - **Wind Features:**

     - Converts the 3–4 pm wind direction from degrees to radians and decomposes the wind speed into its x and y components. The temporary column used for conversion is dropped afterward.

   - **Temporal Anomalies:**

     - For each weather parameter measured between 3–4 pm, the code computes an anomaly by subtracting the overall full‐period average from the 3–4 pm average.

   - **Reference ET0 Calculation:**

     - Using the FAO‑56 Penman–Monteith equation, the code computes reference evapotranspiration (ET0) from air temperature, wind speed, solar flux, relative humidity, and albedo. This gives an estimate of water demand that is important in understanding surface energy balance.

   - **Urban & Building Characteristics:**
     - **Normalized Distance To Green:** Ratio of NDVI-based distance to total area.
     - **Building Density Gradient:** Uses a spatial gradient (via `np.gradient`) of building counts to capture spatial variations in density.
     - **Road Network Clustering:** Applies Kernel Density Estimation (KDE) on the road network to compute a clustering index.
     - **LST Variability:** Estimates the spatial variability (standard deviation) of LST using a KDTree approach.
     - **Urbanization Intensity:** Combines normalized road density, building counts, and vegetation index (with an inversion of NDVI) to provide a composite index of urbanization.
     - **Building Compactness:** Ratio of total building area to the product of building count and road density.
     - **Green Coverage Ratio:** Relates the NDVI distance metric to building counts.
     - **LST Anomaly:** Computes the deviation of each point’s LST from the overall mean.
     - **Building Coverage Ratio:** Ratio of building area to the sum of building area and NDVI distance.
     - **LST Difference from Reference:** Computes the difference between a point’s LST and the reference LST (including a power transformation for nonlinearity).
     - **Urban Core Compactness:** Combines building compactness and urbanization intensity normalized by road density.
     - **Advanced Green Space Metrics:**
       - **Vegetation Density Gradient:** Spatial gradient of NDVI.
       - **Green Heat Absorption Index:** Ratio of green coverage to LST.
     - **Water & Thermal Regulation Features:**
       - **Water Influence Index:** Ratio of NDWI to NDVI distance.
       - **Thermal Stability Index:** Ratio of LST anomaly to urbanization intensity.
     - **Other Interaction Features:**
       - **Vegetation Buffer Effect, Urban Heat Retention Index, Thermal Buffer Effect, Road Density Weighted Index, Infrastructure Coverage Ratio, and Green Space Thermal Impact:** These features combine multiple variables to capture complex interactions influencing the urban heat environment.
   - **Radiation Features:**
     - **Radiation Balance:** Computes the net radiation by adding surface net solar and thermal radiation and then subtracting the latent and sensible heat fluxes.
     - **Effective Solar:** Adjusts the solar radiation by the Sky View Factor (SVF) to account for shading.
     - **Additional Interaction Features:**
       - Interactions between solar radiation, temperature, and wind (e.g., `slhf_temp`, `slhf_wind`, `stl_temp`, `temp_radiation`) are calculated to capture combined effects on the thermal environment.

3. **Application to Datasets:**
   - Finally, the function `engineer_new_features` is applied to both the training and submission datasets so that the model can leverage these newly engineered features.


In [4]:
# Reference LST is obtained from a specific location (using defined longitude and latitude)
reference_long = -73.97277667
reference_lat = 40.76897333
# Get the median LST value from processed_train for the reference location
reference_LST = processed_train[(processed_train["Longitude"] == reference_long) & (processed_train["Latitude"] == reference_lat)]["LST_ls_median"]
reference_LST = float(reference_LST)

def engineer_new_features(df):
    """Generate additional engineered features for UHI (Urban Heat Island) prediction."""

    # Log feature: NDVI_solar
    print("Calculating NDVI_solar")
    # 1️⃣ NDVI_solar:
    # Combine vegetation index (NDVI_median) with solar flux during 3-4pm (w_solar_3_4_mean)
    # to gain insight into surface shading.
    df["NDVI_solar"] = df["NDVI_median"] * df["w_solar_3_4_mean"]

    # Log feature: effective_solar_flux
    if "buildings_count" in df.columns:
        print("Calculating effective_solar_flux")
        # 2️⃣ Effective Solar Flux:
        # Combine solar flux with building density if available. Lower effective solar flux
        # may indicate higher shading or canopy cover.
        df["effective_solar_flux"] = df["w_solar_3_4_mean"] / (1 + df["buildings_count"])

    # ---------------------------
    # TEMPERATURE & HUMIDITY FEATURES
    # ---------------------------

    # Log feature: hum_temp
    print("Calculating hum_temp")
    # 3️⃣ Humidity-Temperature Interaction:
    # This product can hint at the “feels like” temperature.
    df["hum_temp"] = df["w_air_3_4_mean"] * df["w_relative_3_4_mean"] / 100

    # ---------------------------
    # WIND FEATURES
    # ---------------------------

    # Log feature: w_wind_3_4_avg_rad
    print("Converting wind direction to radians (w_wind_3_4_avg_rad)")
    # 4️⃣ Convert wind direction from degrees to radians for further computations.
    df["w_wind_3_4_avg_rad"] = np.deg2rad(df["w_wind_3_4_mean"])

    # Log feature: wind_speed_x and wind_speed_y
    print("Calculating wind_speed_x and wind_speed_y")
    # 5️⃣ Decompose wind into its x and y components.
    df["wind_speed_x"] = df["w_avg_3_4_mean"] * np.cos(df["w_wind_3_4_avg_rad"])
    df["wind_speed_y"] = df["w_avg_3_4_mean"] * np.sin(df["w_wind_3_4_avg_rad"])

    # Drop the temporary column used for wind direction conversion.
    df.drop(columns=["w_wind_3_4_avg_rad"], inplace=True)

    # ---------------------------
    # TEMPORAL TRENDS / ANOMALIES
    # ---------------------------

    print("Calculating temporal anomalies for weather parameters")
    # For each weather parameter with a 3-4pm average, compute the anomaly by subtracting the full-period mean.
    for var in df.columns:
        if var.endswith('_3_4_mean'):
            base = var[:-len('_3_4_mean')]
            df[f"{base}_anomaly"] = df[f"{base}_3_4_mean"] - df[f"{base}_mean"]

    # ---------------------------
    # COMPUTE REFERENCE ET0 USING THE FAO-56 PENMAN–MONTEITH FORMULA
    # ---------------------------

    print("Calculating ET0 using FAO-56 Penman–Monteith")
    def compute_et0(row):
        # Use a suffix for potential adjustments (here it is an empty string).
        suffix = ""
        T = row["w_air"+suffix+"_mean"]      # Air temperature in °C
        u2 = row["w_avg"+suffix+"_mean"]       # Wind speed at 2 m (approximated)
        # Convert solar flux from W/m² to MJ/m²/day:
        if suffix != "":
            Rs = row["w_solar"+suffix+"_mean"] * 14*3600/10**6
        else:
            Rs = row["w_solar"+suffix+"_mean"] * 3600/10**6
        RH = row["w_relative"+suffix+"_mean"]  # Relative Humidity in percent

        G = 0.0           # Soil heat flux density (MJ/m²/day), assumed 0 for daily calculations
        P = 101.3         # Atmospheric pressure in kPa (assumed sea level)

        # Approximate net radiation (Rn) using the albedo correction:
        Rn = (1 - row["albedo_corr_median"]) * Rs  # Net radiation (MJ/m²/day)

        # Compute saturation vapor pressure (es) using temperature (°C)
        es = 0.6108 * np.exp((17.27 * T) / (T + 237.3))

        # Compute slope of the saturation vapor pressure curve (Δ)
        delta = (4098 * es) / ((T + 237.3) ** 2)

        # Compute actual vapor pressure (ea) using relative humidity
        ea = (RH / 100.0) * es

        # Psychrometric constant (γ)
        gamma = 0.665e-3 * P

        # FAO-56 Penman–Monteith Equation to estimate reference ET0 (mm/day)
        ET0 = (0.408 * delta * (Rn - G) + gamma * (900 / (T + 273)) * u2 * (es - ea)) / \
              (delta + gamma * (1 + 0.34 * u2))
        return ET0

    # Compute ET0 for each row and add it as a new column.
    df["ET0"] = df.apply(compute_et0, axis=1)

    # ---------------------------
    # ADDITIONAL URBAN HEAT ISLAND RELATED FEATURES
    # ---------------------------

    print("Calculating Normalized_Distance_To_Green")
    # 1️⃣ Normalized Distance to Green Space:
    # Ratio of NDVI distance to total area (avoid division by zero with small constant).
    df["Normalized_Distance_To_Green"] = df["NDVI_distance_to_high_median"] / (df["area_total"] + 1e-10)

    print("Calculating Building_Density_Gradient")
    # 2️⃣ Building Density Gradient:
    # Compute the gradient (spatial change) in building count.
    df["Building_Density_Gradient"] = np.gradient(df["buildings_count"])

    print("Calculating Road_Clustering_Index")
    # 3️⃣ Road Network Clustering Index:
    # Use Kernel Density Estimation (KDE) to measure the clustering of road networks.
    kde = KernelDensity(kernel="gaussian", bandwidth=500).fit(df[["Longitude", "Latitude"]])
    df["Road_Clustering_Index"] = np.exp(kde.score_samples(df[["Longitude", "Latitude"]]))

    print("Calculating LST_Variability")
    # 4️⃣ LST Variability:
    # Compute the spatial variability (standard deviation) of Land Surface Temperature (LST) using a KDTree.
    def compute_lst_variability(coords, lst_values, radius=500):
        tree = cKDTree(coords)
        _, idx = tree.query(coords, k=10)
        variability = np.array([np.std(lst_values[i]) for i in idx])
        return variability

    coords = df[["Longitude", "Latitude"]].values
    df["LST_Variability"] = compute_lst_variability(coords, df["LST_ls_median"].values)

    print("Calculating Urbanization_Intensity")
    # 5️⃣ Urbanization Intensity Index:
    # Normalize road density, building count, and vegetation indices, then combine them.
    df['R_norm'] = (df['road_density'] - df['road_density'].min()) / (df['road_density'].max() - df['road_density'].min())
    df['B_norm'] = (df['buildings_count'] - df['buildings_count'].min()) / (df['buildings_count'].max() - df['buildings_count'].min())
    df['V_norm'] = (df['NDVI_median'] - df['NDVI_median'].min()) / (df['NDVI_median'].max() - df['NDVI_median'].min())
    df["Urbanization_Intensity"] = df["R_norm"] + df["B_norm"] + (1 - df["V_norm"])
    df.drop(columns=["R_norm", "B_norm", "V_norm"], inplace=True)

    print("Calculating Building_Compactness")
    # 6️⃣ Building Compactness Index:
    # Ratio of total building area to the product of building count and road density.
    df["Building_Compactness"] = df["area_total"] / (df["buildings_count"] * df["road_density"] + 1e-10)

    print("Calculating Green_Coverage_Ratio")
    # 7️⃣ Green Coverage Ratio:
    df["Green_Coverage_Ratio"] = df["NDVI_distance_to_high_median"] / (1 + df["buildings_count"])

    print("Calculating LST_Anomaly")
    # 8️⃣ Temperature Difference from Mean LST (LST Anomaly):
    df["LST_Anomaly"] = df["LST_ls_median"] - df["LST_ls_median"].mean()

    print("Calculating Building_Coverage_Ratio")
    # 9️⃣ Building Coverage Ratio:
    df["Building_Coverage_Ratio"] = df["area_total"] / (df["area_total"] + df["NDVI_distance_to_high_median"] + 1e-10)

    print("Calculating LST_ref_diff and LST_ref_diff_pow")
    # Compute the LST difference relative to the reference LST defined above.
    df["LST_ref_diff"] = df["LST_ls_median"] - reference_LST

    # Apply a power transformation (3/4 exponent) to the LST difference.
    df["LST_ref_diff_pow"] = (df["LST_ref_diff"])**(3/4)

    print("Calculating Urban_Core_Compactness")
    # 1️⃣ Urban Heat & Compactness Features:
    df["Urban_Core_Compactness"] = (df["Building_Compactness"] * df["Urbanization_Intensity"]) / (df["road_density"] + 1e-10)

    print("Calculating Vegetation_Density_Gradient and Green_Heat_Absorption_Index")
    # 2️⃣ Advanced Green Space Metrics:
    df["Vegetation_Density_Gradient"] = np.gradient(df["NDVI_median"])
    df["Green_Heat_Absorption_Index"] = df["Green_Coverage_Ratio"] / (df["LST_ls_median"] + 1e-10)

    print("Calculating Water_Influence_Index and Thermal_Stability_Index")
    # 3️⃣ Water & Thermal Regulation Features:
    df["Water_Influence_Index"] = df["NDWI_median"] / (df["NDVI_distance_to_high_median"] + 1e-10)
    df["Thermal_Stability_Index"] = df["LST_Anomaly"] / (df["Urbanization_Intensity"] + 1e-10)

    print("Calculating Vegetation_Buffer_Effect")
    # Vegetation Buffer Effect:
    df["Vegetation_Buffer_Effect"] = df["NDVI_median"] / (df["NDVI_distance_to_high_median"] + 1e-10)

    print("Calculating Urban_Heat_Retention_Index")
    # Urban Heat Retention Index:
    df["Urban_Heat_Retention_Index"] = df["LST_ls_median"] / (df["Urbanization_Intensity"] + 1e-10)

    print("Calculating Thermal_Buffer_Effect")
    # 2️⃣ Thermal Buffer Effect (TBE):
    df["Thermal_Buffer_Effect"] = df["NDVI_median"] / (df["LST_Variability"] + 1e-10)

    print("Calculating road_density_Weighted_Index")
    # 3️⃣ Road Density Weighted Index (RDWI):
    df["road_density_Weighted_Index"] = df["road_density"] * df["buildings_count"]

    print("Calculating Infrastructure_Coverage_Ratio")
    # 4️⃣ Infrastructure Coverage Ratio (ICR):
    df["Infrastructure_Coverage_Ratio"] = df["Building_Coverage_Ratio"] / (df["Road_Clustering_Index"] + 1e-10)

    print("Calculating Green_Space_Thermal_Impact")
    # 5️⃣ Green Space Thermal Impact (GSTI):
    df["Green_Space_Thermal_Impact"] = df["Green_Coverage_Ratio"] / (df["LST_ls_median"] + 1e-10)

    print("Calculating Water_Influence_Index (WII)")
    # 6️⃣ Water Influence Index (WII):
    df["Water_Influence_Index"] = df["NDWI_median"] / (df["NDVI_distance_to_high_median"] + 1e-10)

    print("Calculating radiation_balance, adjusted_solar, effective_solar")
    # Additional interactions involving albedo and radiation variables:
    df["radiation_balance"] = (df["ssr_mean"] + df["str_mean"] - (df["slhf_mean"] + df["sshf_mean"]))
    df["adjusted_solar"] = df["ssrd_mean"] * (1 - df["fal_mean"])

    # Incorporate interactions with the Sky View Factor (SVF)
    df["effective_solar"] = df["adjusted_solar"] * df["SVF"]

    print("Calculating slhf_temp, slhf_wind, stl_temp, temp_radiation")
    # Incorporate interations between wind, temperature and radiantion features
    df["slhf_temp"] =(df["slhf_3_4_min"]) * df["t2m_3_4_mean"]
    df["slhf_wind"] =(df["slhf_3_4_min"]) * df["v10_3_4_mean"]
    df["stl_temp"] =(df["stl4_3_4_std"]) /  df["t2m_3_4_mean"]
    df["temp_radiation"] =(df["t2m_3_4_mean"]) * ((df["radiation_balance"])**(3/4))

    return df

# Apply feature engineering to both training and submission datasets.
processed_train = engineer_new_features(processed_train.copy())
processed_sub = engineer_new_features(processed_sub.copy())

# Define a function to get columns where all values are constant.
def get_constant_columns(df):
    """Returns a list of column names where all values are the same."""
    return df.columns[df.nunique() == 1].tolist()

# Compare constant columns between processed_train and processed_sub.
if get_constant_columns(processed_train) == get_constant_columns(processed_sub):
    # Drop the constant columns from both datasets if they match.
    print("Dropping constant columns")
    processed_train = processed_train.drop(columns=get_constant_columns(processed_train))
    processed_sub = processed_sub.drop(columns=get_constant_columns(processed_sub))
else:
    print("Columns to drop do not match")

Calculating NDVI_solar
Calculating effective_solar_flux
Calculating hum_temp
Converting wind direction to radians (w_wind_3_4_avg_rad)
Calculating wind_speed_x and wind_speed_y
Calculating temporal anomalies for weather parameters
Calculating ET0 using FAO-56 Penman–Monteith
Calculating Normalized_Distance_To_Green
Calculating Building_Density_Gradient
Calculating Road_Clustering_Index
Calculating LST_Variability
Calculating Urbanization_Intensity
Calculating Building_Compactness
Calculating Green_Coverage_Ratio
Calculating LST_Anomaly
Calculating Building_Coverage_Ratio
Calculating LST_ref_diff and LST_ref_diff_pow
Calculating Urban_Core_Compactness
Calculating Vegetation_Density_Gradient and Green_Heat_Absorption_Index
Calculating Water_Influence_Index and Thermal_Stability_Index
Calculating Vegetation_Buffer_Effect
Calculating Urban_Heat_Retention_Index
Calculating Thermal_Buffer_Effect
Calculating road_density_Weighted_Index
Calculating Infrastructure_Coverage_Ratio
Calculating Gre

# Model Development


**Overview:**  
In this section, we prepare our processed data for modeling, optimize the model parameters using Bayesian optimization (Optuna), and then train an XGBoost regression model using the best saved parameters. The goal is to predict the Urban Heat Island (UHI) Index based on a wide range of engineered features.

**Input:**

- Processed training and submission datasets containing engineered environmental, urban, and meteorological features.
- A target variable ("UHI Index") for the training data.
- A JSON file with the best hyperparameters obtained from previous model optimization.

**Output:**

- Preprocessed, scaled, and aligned feature matrices for both training and submission.
- An XGBoost regression model trained using the optimized hyperparameters.
- Performance metrics (R² scores) on both training and test splits to assess the model's predictive performance.

**Summary:**

1. **Data Preparation and Scaling:**  
   The code creates copies of the processed datasets, drops spatial and datetime-related columns, and aligns the training and submission feature sets. It then separates the target variable from the predictors and scales the features using StandardScaler. A train–test split is performed to enable model evaluation.

2. **Model Optimization (Optional):**  
   A Bayesian optimization routine (via Optuna) is defined to search for the best hyperparameters for the XGBoost model using cross-validation. The best parameters are saved in a JSON file for later use.

3. **XGBoost Modeling:**  
   Using the saved best parameters, an XGBoost regression model is initialized and trained on the scaled training data. Finally, the model's performance is evaluated on both training and test sets, and the R² scores are printed.

**Features Generated (for Modeling):**

- Engineered features capturing various aspects of urban heat (e.g., vegetation indices, solar flux, temperature-humidity interactions, wind components).
- Derived temporal anomalies comparing 3–4 pm averages against full-period means.
- Complex urban metrics including building compactness, road clustering, and urbanization intensity.
- Additional features incorporating interactions (e.g., albedo with solar flux and sky view factors).


## Data Preparation and Scaling


In [5]:
# Create copies of the processed training and submission datasets.
train_feat = processed_train.copy()
sub_feat = processed_sub.copy()

# Define columns to drop from the datasets (e.g., spatial or datetime columns not needed for modeling).
cols_to_drop = ['Longitude', 'Latitude', "datetime", "geometry", "centroid"]

# Drop the defined columns from the training dataset.
model_train = train_feat.drop(columns=cols_to_drop, errors="ignore")
# Optionally, you can drop rows with missing values (commented out).
# model_train = model_train.dropna()

# Drop the defined columns from the submission dataset.
model_sub = sub_feat.drop(columns=cols_to_drop, errors="ignore")

# Align the submission DataFrame to have the same columns as the training DataFrame.
model_sub = model_sub[model_train.columns]

# Separate the features (X) and the target variable (y) from the training data.
# Here, "UHI Index" is the target variable.
X_full = model_train.drop(columns=["UHI Index"]).values
y_full = model_train["UHI Index"].values

# Split the training data into a training set and a test set for GridSearchCV evaluation.
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=123)

# Scale the features using StandardScaler:
# Fit the scaler on the training data and transform both training and test sets.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



## Model optimization


In [6]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=25),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 0.5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 100, log=True),
    }

    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **params)
    score = cross_val_score(model, X_train, y_train, scoring="r2", cv=5, n_jobs=-1).mean()
    return score

# Run Bayesian Optimization
optimize = False
if optimize:
    study = optuna.create_study(direction="maximize")  # maximize negative MSE (minimize MSE)
    study.optimize(objective, n_trials=1000)

    # Save the best parameters
    xgb_optuna = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **study.best_params)

    def save_params(model):
    # Save to a JSON file
        print("Saving the best model parameters...")
        with open(os.path.join("Output", "best_model_params.json"), "w") as f:
            json.dump(model.get_params(), f, indent=4)
        print("Best model saved")

    save_params(xgb_optuna)

## Xgboost modeling


**Feature Selection & Dropping Rationale**

In our modeling process, we conducted an in-depth analysis to refine the set of features used for predicting the Urban Heat Island (UHI) Index. This selection was driven by two main factors:

1. **XGBoost Feature Importance Analysis:**  
   We ran an initial XGBoost model and examined the feature importance results. Features that showed little or no contribution to the model’s predictive performance were flagged as candidates for removal. This analysis helped us identify redundant or non-informative features.

2. **Domain Expertise and Experimental Insights:**  
   Based on our own observations, additional experiments, and valuable insights from domain experts we consulted, we determined that several features (especially those related to raw spatial coordinates and certain complex derivative metrics) did not add meaningful predictive power or could potentially introduce noise and overfitting into the model.

As a result, we dropped:

- **Low-Importance and Redundant Features:**  
  The extensive list of features in `features_to_drop` (derived from both the feature importance analysis and expert feedback) were excluded to simplify the model and focus on the most relevant predictors.

This careful feature selection aims to improve model performance, enhance interpretability, and reduce the risk of overfitting.


In [7]:
cols_to_drop = ['Longitude', 'Latitude', "datetime", "geometry", "centroid"]
features_to_drop = ['B11_median_1', 'fal_max', 'LST_ls_max_1', 'nearest_building_compactness', 'slhf_3_4_std', 'sp_3_4_mean', 'stl4_min', 'B11_median_2', 'NDVI_distance_to_high_median_3', 'w_solar_3_4_min', 'NDVI_buffer_std_median_2', 'v10_3_4_mean', 'sp_max', 'LST_Variability', 'NDVI_median_3', 'density_min', 'sshf_min', 'w_wind_anomaly', 'ssrd_anomaly', 'w_density_mean', 'stl3_min', 'NDRE_median_3', 'v10_std', 'stl3_std', 'w_solar_anomaly', 'NDMI_min_4', 'u10_3_4_min', 'LST_ls_min', 'w_relative_3_4_mean', 'w_ke_3_4_std', 'ssrd_3_4_min', 'w_density_3_4_mean', 'w_air_3_4_std', 'd2m_anomaly', 'w_density_3_4_min', 'sshf_anomaly', 'SAVI_min_4', 'NDWI_distance_to_high_median', 'strd_3_4_mean', 'sshf_gradient', 'wind_dir_3_4_min', 'LST_Anomaly', 'road_density_Weighted_Index', 'B01_median_3', 'NDWI_max_3', 'BSI_median_1', 'w_vpd_3_4_min', 'density_std', 'sshf_3_4_max', 'thermal_gradient_median_1', 'ro_anomaly', 'nearest_building_perimeter', 'w_relative_std', 'w_heat_max', 'w_air_3_4_min', 'str_3_4_min', 'w_ke_3_4_max', 'w_cooling_3_4_mean', 'Road_Clustering_Index', 'SVF', 'LST_ls_min_1', 'B05_median_3', 'B06_median_2', 'skt_min', 'NDMI_median_2', 'SAVI_median_4', 'Urban_Core_Compactness', 'w_wind_3_4_max', 'NDRE_median', 'wind_dir_3_4_std', 'inside_building', 'NDWI_buffer_std_median_2', 'SAVI_min_1', 'NDWI_max', 'w_wind_3_4_mean', 'ssr_3_4_mean', 'ro_3_4_std', 'NDVI_max_2', 'NDRE_max_3', 'w_air_3_4_max', 'LST_ls_max', 't2m_max', 'Green_Space_Thermal_Impact', 'B12_median', 'u10_min', 'v10_3_4_min', 'sshf_std', 'fal_slope', 'NDMI_max', 'NDVI_solar', 'LST_ls_median_2', 'NDMI_median_4', 'SAVI_median_2', 'w_ke_3_4_min', 'w_density_max', 'sshf_mean', 'NDVI_max_4', 'w_density_min', 'B07_median_1', 'slhf_3_4_max', 'w_dew_mean', 'd2m_3_4_max', 'density_avg', 'u10_anomaly', 'height_std', 'B12_median_2', 'B05_median', 'stl1_min', 'NDWI_max_4', 'w_avg_anomaly', 'B07_median_4', 'NDWI_median_4', 'stl3_3_4_std', 'buildings_count', 'NDVI_buffer_std_median_1', 'tp_second_deriv', 'u10_3_4_std', 'NDVI_buffer_std_median_3', 'stl4_3_4_mean', 'NDVI_max', 'FAR', 'NDWI_median_1', 'B8A_median_1', 'SAVI_max_4', 'w_wind_min', 'NDVI_distance_to_high_median', 'density_median', 'str_3_4_max', 'Thermal_Buffer_Effect', 'w_vpd_3_4_max', 'w_avg_3_4_std', 'w_dew_3_4_std', 'BSI_median_2', 'NDRE_max_2', 'skt_3_4_mean', 'NDWI_buffer_std_median_4', 'w_air_anomaly', 'B06_median_3', 'w_vpd_mean', 'w_air_mean', 'stl1_3_4_max', 'w_heat_min', 'effective_solar', 'wind_dir_mean', 'B06_median_4', 'albedo_corr_median_4', 'w_dew_3_4_max', 'NDVI_median_4', 'ro_3_4_mean', 'effective_solar_flux', 'tp_mean', 'w_cooling_3_4_std', 'Infrastructure_Coverage_Ratio', 'thermal_gradient_median_3', 'stl4_max', 'B8A_median_2', 'w_wind_std', 'w_avg_3_4_mean', 'stl1_3_4_mean', 'ssrd_min', 'perimeter_std', 'tp_3_4_mean', 'ssrd_mean', 'NDVI_max_3', 'thermal_gradient_median_2', 'area_std', 'thermal_gradient_median', 'LST_ls_max_2', 'w_solar_max', 'fal_second_deriv', 'strd_3_4_min', 'w_cooling_3_4_max', 'd2m_3_4_mean', 't2m_min', 'NDVI_buffer_std_median_4', 'Green_Heat_Absorption_Index', 'str_3_4_mean', 'height_avg', 'compactness_max', 'tp_gradient', 'NDVI_min_2', 'NDWI_median', 'B07_median_3', 'fal_mean', 'skt_3_4_min', 'Urbanization_Intensity', 't2m_mean', 'w_ke_anomaly', 't2m_3_4_max', 'NDWI_buffer_std_median_3', 'w_vpd_anomaly', 'NDVI_min', 'Urban_Heat_Retention_Index', 'BCR', 'B01_median_1', 'NDMI_max_4', 'w_vpd_std', 'LST_ls_median_1', 'SAVI_min_3', 'stl4_mean', 'NDMI_max_1', 'w_relative_3_4_std', 'NDWI_median_2', 'NDWI_min', 'NDRE_max_1', 'd2m_min', 'SAVI_max_1', 'strd_mean', 'stl2_max', 'w_air_min', 'v10_min', 'w_cooling_mean', 'BSI_median_3', 'fal_anomaly', 'w_wind_mean', 'ssr_mean', 'w_heat_3_4_min', 'stl1_std', 'nearest_building_area', 'stl3_mean', 'NDWI_buffer_std_median_1', 'compactness_std', 'd2m_std', 'LST_ls_max_4', 'fal_min', 'area_total', 'skt_3_4_max', 'tp_slope', 'w_vpd_max', 'w_ke_mean', 'LST_ls_median_3', 'NDMI_max_3', 'Vegetation_Buffer_Effect', 'dist_to_nearest_building', 'w_solar_std', 'NDMI_min_2', 'w_solar_min', 'NDVI_min_1', 'u10_3_4_mean', 'w_avg_min', 'u10_3_4_max', 'B12_median_4', 'B8A_median_3', 'NDWI_buffer_std_median', 'ro_std', 'nearest_building_feat_code', 'LST_ls_min_3', 'NDRE_min', 'w_avg_3_4_max', 'w_avg_std', 'ssr_3_4_min', 'w_relative_anomaly', 'w_relative_3_4_max', 't2m_std', 'B01_median_4', 'compactness_avg', 'slhf_std', 'w_air_std', 'w_vpd_3_4_mean', 'B11_median_3', 'B8A_median', 'sp_min', 'NDMI_min', 'B01_median', 'tp_3_4_min', 'ssrd_std', 'v10_max', 'NDMI_median', 'wind_dir_std', 'tp_std', 'stl3_3_4_min', 'w_dew_max', 'thermal_gradient_median_4', 'w_cooling_min', 'w_air_max', 'albedo_corr_median_1', 'B06_median_1', 'w_dew_min', 'stl1_anomaly', 'strd_anomaly', 'NDMI_median_1', 'LST_ref_diff_pow', 'compactness_median', 'Thermal_Stability_Index', 'perimeter_avg', 'B11_median_4', 'NDRE_median_2', 'ro_3_4_min', 'NDRE_median_1', 'Normalized_Distance_To_Green', 'w_heat_3_4_mean', 'LST_ls_median_4', 'w_cooling_3_4_min', 'w_heat_std', 'NDMI_min_1', 'w_avg_max', 'stl3_3_4_mean', 'NDVI_max_1', 'ssrd_3_4_mean', 'w_ke_min', 'LST_ref_diff', 'w_dew_anomaly', 'BSI_median', 'hum_temp', 'perimeter_median', 'NDVI_distance_to_high_median_2', 'albedo_corr_median_2', 'str_max', 'v10_3_4_max', 'fal_3_4_min', 'Vegetation_Density_Gradient', 'NDRE_max_4', 'strd_min', 'd2m_3_4_std', 'Building_Compactness', 'w_cooling_std', 'v10_3_4_std', 'w_heat_3_4_max', 'albedo_corr_median', 'area_avg', 'NDWI_min_1', 'ET0', 'w_cooling_max', 'strd_3_4_max', 'Green_Coverage_Ratio', 'SAVI_min', 'w_solar_mean', 'w_ke_std', 'ssrd_3_4_max', 'w_solar_3_4_max', 'Water_Influence_Index', 'w_density_3_4_std', 'NDWI_max_1', 'w_wind_3_4_min', 'wind_dir_3_4_mean', 'stl4_std', 'w_avg_3_4_min', 'NDVI_distance_to_high_median_4', 'area_median', 'NDVI_distance_to_high_median_1', 'fal_3_4_mean', 'NDWI_min_4', 'w_density_anomaly', 'Building_Coverage_Ratio', 'SAVI_median', 'B05_median_2', 'BSI_median_4', 'wind_dir_min', 'ro_min', 'stl2_3_4_max', 'wind_dir_3_4_max', 'slhf_anomaly', 'w_solar_3_4_mean', 'w_relative_mean', 'NDRE_min_3', 'wind_speed_x', 'B12_median_1', 'ro_3_4_max', 'wind_dir_max', 'ssr_min', 'NDRE_median_4', 'NDMI_min_3', 'd2m_3_4_min', 'B05_median_1', 'w_dew_std', 'w_dew_3_4_min', 'skt_3_4_std', 'NDWI_max_2', 'B8A_median_4', 'stl2_mean', 'w_ke_max', 'w_heat_3_4_std', 'sp_mean', 'ssr_3_4_max', 'ssr_3_4_std', 'stl3_anomaly', 'SAVI_median_3', 'str_3_4_std', 'SAVI_max_2', 'stl2_std', 'stl2_3_4_std', 'NDRE_min_4', 'skt_max', 'w_vpd_min', 'NDRE_max', 'B05_median_4', 'stl2_3_4_min', 'tp_3_4_max', 'B07_median_2', 'fal_3_4_max', 'B01_median_2', 'NDMI_max_2', 'w_relative_3_4_min', 'w_relative_min', 'slhf_min', 'stl4_anomaly', 'w_density_3_4_max', 'sp_3_4_std', 'str_mean', 'SAVI_min_2', 'u10_mean', 'NDRE_min_1', 'stl3_3_4_max', 'slhf_3_4_min', 'stl3_max', 'B11_median', 'wind_dir_anomaly', 'w_relative_max', 'SAVI_max', 'NDWI_distance_to_high_median_2', 'LST_ls_min_2', 'height_median', 'w_avg_mean', 'NDVI_median', 'area_min', 'w_cooling_anomaly', 'w_dew_3_4_mean', 'str_std', 'stl4_3_4_max', 'stl1_3_4_min', 'NDVI_median_1', 'Building_Density_Gradient', 'sp_anomaly', 'ssrd_max', 'w_wind_3_4_std', 'strd_3_4_std', 'NDWI_distance_to_high_median_1', 'NDRE_min_2', 'SAVI_max_3', 'B06_median', 'w_vpd_3_4_std', 'ssrd_3_4_std', 'stl2_3_4_mean', 'w_heat_anomaly', 'v10_slope', 'NDVI_median_2', 'B07_median', 'w_wind_max', 'LST_ls_max_3', 'SAVI_median_1', 'skt_std', 'B12_median_3', 'w_ke_3_4_mean', 'NDWI_min_2', 'w_heat_mean', 'BD', 'u10_max', 't2m_3_4_std', 'sshf_3_4_min', 'NDVI_min_3', 't2m_3_4_min', 'LST_ls_median', 'wind_speed_y', 'stl1_mean', 'NDMI_median_3', 'NDVI_buffer_std_median', 'NDWI_median_3', 'ssr_std', 'w_air_3_4_mean', 'u10_std', 'road_density', 'NDVI_min_4', 'fal_gradient', 'stl1_3_4_std', 'NDWI_min_3', 'nearest_building_height', 'albedo_corr_median_3', 'stl4_3_4_min']
features_to_drop += ['sp_slope', 'sp_gradient', 'sp_second_deriv', 'u10_slope', 'u10_gradient', 'u10_second_deriv', 't2m_slope', 't2m_gradient', 't2m_second_deriv', 'd2m_slope', 'd2m_gradient', 'd2m_second_deriv', 'ssrd_slope', 'ssrd_gradient', 'ssrd_second_deriv', 'strd_slope', 'strd_gradient', 'strd_second_deriv', 'ssr_slope', 'ssr_gradient', 'ssr_second_deriv', 'str_slope', 'str_gradient', 'str_second_deriv', 'ro_slope', 'ro_gradient', 'ro_second_deriv', 'stl1_slope', 'stl1_gradient', 'stl1_second_deriv', 'stl2_slope', 'stl2_gradient', 'stl2_second_deriv', 'stl3_slope', 'stl3_gradient', 'stl3_second_deriv', 'stl4_slope', 'stl4_gradient', 'stl4_second_deriv']

for col in processed_train.columns:
    if "nearest_high_median" in col:
        features_to_drop.append(col)
    if "buffer_high_median" in col:
        features_to_drop.append(col)
    if "NDVI_ls_median" in col:
        features_to_drop.append(col)


features_to_drop += ["albedo_ndvi", "albedo_effect", "adjusted_solar", "thermal_diff", "latent_sensible_ratio", "radiation_balance_svf", "albedo_svf_interaction"]

In [8]:
# Drop the defined columns from the training dataset.
model_train = train_feat.drop(columns= cols_to_drop + features_to_drop, errors="ignore")
# Optionally, you can drop rows with missing values (commented out).
# model_train = model_train.dropna()

# Drop the defined columns from the submission dataset.
model_sub = sub_feat.drop(columns=cols_to_drop + features_to_drop, errors="ignore")

# Align the submission DataFrame to have the same columns as the training DataFrame.
model_sub = model_sub[model_train.columns]

# Separate the features (X) and the target variable (y) from the training data.
# Here, "UHI Index" is the target variable.
X_full = model_train.drop(columns=["UHI Index"]).values
y_full = model_train["UHI Index"].values

# Split the training data into a training set and a test set for GridSearchCV evaluation.
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=123)

# Scale the features using StandardScaler:
# Fit the scaler on the training data and transform both training and test sets.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
is_train = False
if is_train :
  # import json file of the best parameters
  saved_best_params = json.load(open(os.path.join("data", "best_model_params.json"), "r"))

  # --- Initialize and train the model with saved best parameters ---

  # Assume saved_best_params is a dictionary with the best hyperparameters
  model_optuna = xgb.XGBRegressor(**saved_best_params)
  # commented code: if we decide to use directly the output of optuna optimization instead of loadig the best model parameters

  model_optuna.fit(X_train_scaled, y_train)

  # Evaluate on training and test splits
  train_r2 = r2_score(y_train, model_optuna.predict(X_train_scaled))
  test_r2  = r2_score(y_test, model_optuna.predict(X_test_scaled))
  print("Training R² on split:", train_r2)
  print("Test R² on split:", test_r2)


# Model Inference


In this section, we finalize our modeling process by retraining the model using the full training dataset and generating predictions on the submission dataset. The steps include:

1. **Scaling the Full Training Data:**  
   We re-scale the entire training feature set using the same StandardScaler that was fitted during model optimization. This ensures that all features are normalized consistently.

2. **Model Retraining:**  
   The XGBoost model (with the best hyperparameters found during optimization) is re-trained on the scaled full training data. This leverages all available training samples for the final model.

3. **Submission Predictions:**  
   The submission dataset's features are scaled with the same scaler, and the retrained model is then used to generate predictions for the target variable ("UHI Index").

4. **Preparing the Submission File:**  
   Predictions are inserted into a submission template, and (if saving is enabled) the final submission file is saved as a CSV file.


In [18]:
if is_train :
  # --- Re-train the model on the full training dataset ---
  X_full_scaled = scaler.fit_transform(X_full)  # Scale full training data
  model_optuna.fit(X_full_scaled, y_full)

  # --- Predict on the submission dataset ---
  X_submission = scaler.transform(model_sub.drop(columns=["UHI Index"]).values)
  submission_predictions_xgb = model_optuna.predict(X_submission)

In [19]:
X_submission = scaler.transform(model_sub.drop(columns=["UHI Index"]).values)


model = xgb.XGBRegressor()
model.load_model(os.path.join("data","my_xgb_model.bin"))
submission_predictions_xgb = model.predict(X_submission)

In [20]:
print(list(submission_predictions_xgb))

[np.float32(0.96378314), np.float32(0.9633674), np.float32(0.96434265), np.float32(0.9615813), np.float32(0.95944405), np.float32(0.95954514), np.float32(0.95895857), np.float32(0.95837504), np.float32(0.9607056), np.float32(0.9620402), np.float32(0.9737504), np.float32(0.97341794), np.float32(0.97080356), np.float32(0.96890837), np.float32(0.9696781), np.float32(0.974681), np.float32(0.96957034), np.float32(0.9705271), np.float32(0.9677654), np.float32(0.96975875), np.float32(0.9671814), np.float32(0.9745463), np.float32(0.97323275), np.float32(0.9731607), np.float32(0.9739379), np.float32(0.9747265), np.float32(0.9741899), np.float32(0.97435606), np.float32(0.9673695), np.float32(0.9719341), np.float32(0.9716274), np.float32(0.9707429), np.float32(0.9748661), np.float32(0.97342426), np.float32(0.97199017), np.float32(0.96815914), np.float32(0.9749553), np.float32(0.9691876), np.float32(0.970059), np.float32(0.97631603), np.float32(0.97024804), np.float32(0.9734251), np.float32(0.9713

In [21]:
submission_df = pd.read_csv(os.path.join("data","Submission_template.csv"))
submission_df["UHI Index"] = submission_predictions_xgb

save = True
if save:

    # Définir le répertoire à parcourir (remplacez par votre chemin)
    submission_df.to_csv(os.path.join("data", "Submission.csv"),index = False)